In [ ]:
# Bibliotecas
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import boto3
import io
import s3fs

In [ ]:
# S3 input
bucket = 'seu-bucket'
path_input = 'caminho/base_input.csv'
s3_uri = f's3://{bucket}/{path_input}'

df = pd.read_csv(s3_uri)

🏗️ 3. Feature Engineering


In [ ]:
# Recência
df['recencia'] = df['dias_ultima_interacao'].apply(lambda x: 1 / (1 + x))

# Modalidade binária
df['pagamento_debito'] = df['modalidade_pagamento'].apply(lambda x: 1 if x == 'debito' else 0)

# Faixa de atraso mapeada
mapa_atraso = {'em dia': 0, '1-15 dias': 1, '16-60 dias': 2, '60+ dias': 3}
df['faixa_atraso'] = df['faixa_atraso'].map(mapa_atraso)

# Penalização por uso indevido da central
df['penalizacao_acionamento'] = df['acionou_canal_para_servico_digital'].apply(lambda x: -0.1 if x == 1 else 0)

📊 4. Normalização e Score

In [ ]:
# Normalização
variaveis = ['qtd_acessos_app', 'nivel_app', 'uso_whatsapp', 'diversidade_servicos',
             'taxa_resolucao_digital', 'recencia', 'tempo_contrato_meses']
scaler = MinMaxScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df[variaveis]), columns=[f'{v}_norm' for v in variaveis])
df = pd.concat([df, df_norm], axis=1)

# Cálculo do Score
def calcular_score(linha):
    if linha['parcela_em_aberto'] == 0:
        return None
    if linha['tipo_cliente'] == 'nao_correntista':
        return max(0, min(1, 0.6 * linha['uso_whatsapp_norm'] +
                               0.4 * linha['taxa_resolucao_digital_norm'] +
                               linha['penalizacao_acionamento']))
    else:
        score = (
            0.15 * linha['qtd_acessos_app_norm'] +
            0.15 * linha['nivel_app_norm'] +
            0.15 * linha['uso_whatsapp_norm'] +
            0.15 * linha['diversidade_servicos_norm'] +
            0.10 * linha['taxa_resolucao_digital_norm'] +
            0.10 * linha['recencia_norm'] +
            0.10 * linha['tempo_contrato_meses_norm'] +
            0.05 * linha['pagamento_debito']
        )
        return max(0, min(1, score + linha['penalizacao_acionamento']))

df['score_digital'] = df.apply(calcular_score, axis=1)

# Classificação
def classificar(score):
    if pd.isnull(score):
        return 'Sem necessidade'
    elif score >= 0.7:
        return 'Alto'
    elif score >= 0.4:
        return 'Médio'
    else:
        return 'Baixo'

df['nivel_digitalizacao'] = df['score_digital'].apply(classificar)

In [1]:
df.head

NameError: name 'df' is not defined

In [ ]:
# Gravação
path_output = 'caminho/digitalizacao_output.parquet'
s3_uri_output = f's3://{bucket}/{path_output}'
df.to_parquet(s3_uri_output, index=False)